In [1]:
from dotenv import load_dotenv
from nltk.twitter.twitter_demo import yesterday

# API 키 정보 로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH11-Retriever")


LangSmith 추적을 시작합니다.
[프로젝트명]
CH11-Retriever


# 낮은 감쇠율(low decay_rate)

In [3]:
from datetime import datetime, timedelta

import faiss
from langchain.docstore import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

# 임베딩 모델을 정의
embeddings_model = OpenAIEmbeddings()

# 벡터 저장소를 빈 상태로 초기화
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})

# 시간 가중치가 적용된 벡터 저장소 검색기를 초기화 (낮은 감쇠율을 적용)
retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=vectorstore,
    decay_rate=.000000000000000000001, k=1
)

In [4]:
# 어제 날짜를 계산
yesterday = datetime.today() - timedelta(days=1)

retriever.add_documents(
    [
        Document(
            page_content="채널을 구독해주세요.",
            metadata={"last_accessed_at": yesterday},
        )
    ]
)

# 다른 문서를 추가. metadata는 별도로 설정하지 않음
retriever.add_documents([Document(page_content="채널 구도 해주실꺼죠? Please!")])

['d50ba5b6-b96b-4beb-afbf-54ed8cc3cdd4']

In [5]:
# 채널을 구독해주세요. 가 가장 먼저 반환되는 이뉴는 가장 두드러지기 때문
# 감쇠율이 0에 가깝기 때문에 여전히 최신 상태를 유지하고 있음을 의미
retriever.invoke("채널")

[Document(metadata={'last_accessed_at': datetime.datetime(2025, 4, 8, 22, 20, 9, 110944), 'created_at': datetime.datetime(2025, 4, 8, 22, 19, 21, 86922), 'buffer_idx': 0}, page_content='채널을 구독해주세요.')]

# 높음 감쇠율(high decay_rate)

In [6]:
# 임베딩 모델을 정의합니다.
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

# 벡터 저장소를 빈 상태로 초기화합니다.
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})

# 시간 가중치가 적용된 벡터 저장소 검색기를 초기화합니다.
retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=vectorstore, decay_rate=0.999, k=1
)

In [7]:
# 어제 날짜를 계산
yesterday = datetime.today() - timedelta(days=1)

retriever.add_documents(
    [
        Document(
            page_content="채널을 구독해주세요.",
            metadata={"last_accessed_at": yesterday},
        )
    ]
)

# 다른 문서를 추가. metadata는 별도로 설정하지 않음
retriever.add_documents([Document(page_content="채널 구도 해주실꺼죠? Please!")])

['8d3dcaca-e189-4fbc-920c-486623e8d5c1']

In [8]:
retriever.invoke("채널")

[Document(metadata={'last_accessed_at': datetime.datetime(2025, 4, 8, 22, 21, 8, 657526), 'created_at': datetime.datetime(2025, 4, 8, 22, 20, 59, 997997), 'buffer_idx': 1}, page_content='채널 구도 해주실꺼죠? Please!')]

# 가상의 시간으로 decay_rate 조정

In [11]:
import datetime

from langchain.utils import mock_now

# 현재 시간 출력
print(datetime.datetime.now())

2025-04-08 22:22:58.078275
